# 深層学習ノートブック-6 誤差逆伝播(Backpropagation)
誤差逆伝播をスクラッチで実装してみる。  
まず大前提として、誤差逆伝播は損失が少なくなるようにパラメータを更新するためのものであるから、  
$\frac{\partial{L}}{\partial{W^{[l]}}}$ と $\frac{\partial{L}}{\partial{B^{[l]}}}$ をいかに求めるかが実装する上で重要なところ。  

* $l$番目の層における損失の誤差 
$$\delta^{[l]}=\frac{\partial{L}}{\partial{Z^{[l]}}}$$  
$$=\frac{\partial{L}}{\partial{A^{[l]}}} \bigodot \frac{\partial{A}}{\partial{Z^{[l]}}}$$
$$=(\delta^{[l+1]}\bm{W}^{[l+1]}) \bigodot \sigma'(\bm{Z^{[l]}})$$  

* パラメタの勾配
$$\frac{\partial{L}}{\partial{W^{[l]}}}=\delta^{[l]T}\bm{A}^{[l-1]}$$  
$$\frac{\partial{L}}{\partial{b^{[l]}}}=\sum_i\delta_i^{[l]}$$  

* パラメタの更新
$$\bm{W^{[l]}}=\bm{W^{[l]}}-\alpha\frac{\partial{L}}{\partial{W^{[l]}}}$$  
$$\bm{B^{[l]}}=\bm{B^{[l]}}-\alpha\frac{\partial{L}}{\partial{B^{[l]}}}$$  


※ReLUの導関数
\begin{equation}
\sigma^{'}(z)= \left \{
\begin{array}{l}
1　(if z > 0) \\
0　(otherwise)
\end{array}
\right.
\end{equation}

In [1]:
import torch
import torch.nn.functional as F  #pytorchの便利関数はFでimportすることが多い。
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# python debugerをインポート
import pdb

## ■ backward（逆伝播）の実装

In [2]:
# 線形変換部分(パラメタ)の勾配の算出関数。Aはl-1層目の値で他はl層目の値。
def linear_backward(A, W, b, Z):
     # pytorch.tensorの.grad属性と区別するために.grad_として属性を定義している
     W.grad_ = Z.grad_.T @ A
     # bの値が一つ変わると、全データ分Lへ影響を及ぼすので、行方向(データ数の方向)にZ.grad_を足す。
     b.grad_ = torch.sum(Z.grad_, dim=0)
     # l-1層目のAはl層目の誤差（=Z.grad_）とl層目のWから計算できる。
     A.grad_ = Z.grad_ @ W


# ReLU関数
def relu_backward(Z, A):
     # 上式の"l番目の層における損失の誤差"における2行目と3行目を組み合わせてdL/dZを算出している
     # A.grad_はdL/dA, (Z > 0).float()はReLu関数の偏微分に相当する。
     Z.grad_ = A.grad_ * ( Z > 0 ).float()

### 補足1
tensor * (tensor > 0)の計算について

In [3]:
a = torch.randn((2,3)) 
b = torch.randn((2,3)) 
print(a)
print(b)
print(b > 0)

tensor([[-0.0382,  1.1134, -1.0477],
        [ 0.1207,  0.5786, -0.2147]])
tensor([[ 1.9893, -0.0070, -0.2763],
        [ 0.3080, -0.6354, -1.0104]])
tensor([[ True, False, False],
        [ True, False, False]])


In [4]:
a * (b > 0)

tensor([[-0.0382,  0.0000, -0.0000],
        [ 0.1207,  0.0000, -0.0000]])

このようにb > 0がTrueの要素は1、Falseの要素は0として計算される。

上記のように損失、パラメタの勾配をコードで表わすことが出来た。  
ではこれらの計算を実際に実行するにはどうすればよいか？  
当然、具体的な計算を行うためには上記の関数の入力であるA,W,b,Zを求める必要がある。  
すなわち順伝播(forward)の計算を行い、損失を計算してから逆伝播(backward)の計算を行うことになる。  

まず損失を求める関数を実装する。

In [5]:
# softmax関数と交差エントロピーの計算を同じ関数で実装する（pytorchでもこうなっている）
def softmax_cross_entropy(X, y_true):
    '''
    X: input tensor.行は各データ、列は各クラスを想定。
    y_true: tensor。One-Hot Encoding済みの正解ラベル。
    '''
    max_val = X.max(dim=1, keepdim=True).values
    # 各要素のe^xを計算（これが分子になる）
    e_x = (X - max_val).exp()
    denominator = e_x.sum(dim=1, keepdim=True) + 1e-10
    softmax_out = e_x / denominator
    loss = - (y_true * torch.log(softmax_out + 1e-10)).sum() / y_true.shape[0]

    return loss, softmax_out


In [6]:
# tensorの線形結合を返す関数
def linear_comb(X, W, B):
    '''
    X, W, B: torch.tensor
    '''
    return X @ W.T + B

In [7]:
# ReLUの実装
def ReLU(Z):
    '''
    Z: torch.tensor
    '''
    # torch.whereによって要素ごとに条件が真・偽のときで別の値を返せる
    # 下記では0より大きい要素はzの値そのままで、0以下は0.になる。
    return torch.where(Z > 0 , Z, 0.)

順伝播→逆伝播の計算を行う関数を定義。  
中間層の勾配もスクラッチの結果と等しいか確認したいので、  
.retain_grad()を次の計算の前に呼び出している。  

In [42]:
# 隠れ層１層の場合のMLP
def forward_and_backward(X, W_list, B_list, y):
    '''
    X: features
    W: List of weights(each edge)
    B: List of Bias(each Layer)
    '''

    #########################
    #### 順伝播（forward）####
    #########################

    # 入力層→隠れ層
    Z1 = linear_comb(X, W_list[0], B_list[0])
    Z1.retain_grad()

    # 活性化関数適用
    A1 = ReLU(Z1)
    A1.retain_grad()

    # 隠れ層→出力層
    Z2 = linear_comb(A1, W_list[1], B_list[1])
    Z2.retain_grad()

    # 最終出力。損失とソフトマックス関数の出力が返される
    loss, A2 = softmax_cross_entropy(Z2, y)


    #########################
    #### 逆伝播（backward）####
    #########################

    # 最終出力→出力層の出力のbackward。ここで各層の出力の勾配を求める。
    # ここでは出力層の活性化関数は恒等関数とし、
    # その出力にsoftmax_cross_entropyを適用した結果をモデルの最終出力として考えている。
    Z2.grad_ = (A2 - y) / X.shape[0]
    # 出力層→隠れ層のbackward
    linear_backward(A1, W_list[1], B_list[1], Z2)
    # 隠れ層の出力側のbackward
    relu_backward(Z1, A1)
    # 隠れ層の入力側のbackward
    linear_backward(X, W_list[0], B_list[0], Z1)
    
    return loss, Z1, A1, Z2, A2
    

### 補足：Z2.grad_をデータ数X.shape[0]で割る理由  
交差エントロピーで算出した全体の損失Lはデータ数で割った平均の形なので、右辺もデータ数で割る必要あり。  
各データに対応する損失をデータ数で割ることで、最終的な全体の損失Lへ与える影響をならしているイメージ。  
誤差逆伝播の講義資料のdL/dZ[2]の式を参照。両辺に1/mが実際はかかっているということ。  

## ■ Autogradの結果と一致することを確認
MNISTデータセットを使って、順伝播後に逆伝播を計算する。  
前処理部分は前回までのコードの使いまわし。

In [49]:
# 変数定義
learning_rate = 0.03
loss_log = []  #学習時の損失記録用のリスト

# データロード
dataset = datasets.load_digits()
feature_names = dataset['feature_names']
X = torch.tensor(dataset['data'], dtype=torch.float32)
target = torch.tensor(dataset['target'])

# shape確認
print(f'shape of X: {X.shape}')
print(X[1])
print('==========================')
print(f'shape of y_train: {target.shape}')
print(target)

# 目的変数のエンコーディング
y_true = F.one_hot(target, num_classes=10)
print(f'shape of y_true: {y_true.shape}')

# 学習データと検証データを8:2に分ける
X_train, X_val, y_train, y_val = train_test_split(X, y_true, test_size=0.2, random_state=42)

print(f'shape of train data: X_train:{X_train.shape}, y_train:{y_train.shape}')
print(f'shape of validation data: X_val:{X_val.shape}, y_val:{y_val.shape}')

# 学習データ・検証データの標準化。検証データの標準化には学習データの平均、標準偏差を使用することに注意。  
X_mean = X_train.mean()
X_std = X_train.std()
X_train = (X_train - X_mean) / X_std
X_val = (X_val - X_mean) / X_std

# データ数、特徴量数、隠れ層のノード数、最終的な出力数（ここではクラス数）を定義
m, nf = X_train.shape
nh = 30
n_out = 10

# 入力層→隠れ層の重みW、バイアス項bの初期化
W_1 = torch.randn(size=(nh, nf) ,requires_grad=True) #出力×入力
B_1 = torch.zeros(size=(1, nh), requires_grad=True) # 1 x 出力

# 隠れ層→出力層の重みW、バイアス項bの初期化
W_2 = torch.randn(size=(n_out, nh) ,requires_grad=True) #出力×入力
B_2 = torch.zeros(size=(1, n_out), requires_grad=True) # 1 x 出力
W_list = [W_1, W_2]
B_list = [B_1, B_2]


shape of X: torch.Size([1797, 64])
tensor([ 0.,  0.,  0., 12., 13.,  5.,  0.,  0.,  0.,  0.,  0., 11., 16.,  9.,
         0.,  0.,  0.,  0.,  3., 15., 16.,  6.,  0.,  0.,  0.,  7., 15., 16.,
        16.,  2.,  0.,  0.,  0.,  0.,  1., 16., 16.,  3.,  0.,  0.,  0.,  0.,
         1., 16., 16.,  6.,  0.,  0.,  0.,  0.,  1., 16., 16.,  6.,  0.,  0.,
         0.,  0.,  0., 11., 16., 10.,  0.,  0.])
shape of y_train: torch.Size([1797])
tensor([0, 1, 2,  ..., 8, 9, 8])
shape of y_true: torch.Size([1797, 10])
shape of train data: X_train:torch.Size([1437, 64]), y_train:torch.Size([1437, 10])
shape of validation data: X_val:torch.Size([360, 64]), y_val:torch.Size([360, 10])


In [50]:
# スクラッチで実装した順伝播→逆伝播の関数を実行。
# これでスクラッチバージョンの勾配(.grad_)が得られる。
loss, Z1, A1, Z2, A2 = forward_and_backward(X_train, W_list=W_list, B_list=B_list, y=y_train)

# autogradバージョンの逆伝播(.grad)を得たいのならば、普通に最終的なlossの結果に.backward()するでOK
loss.backward()

In [51]:
# autogradのdL/dZ2の結果
Z2.grad

tensor([[ 7.5433e-08,  1.5461e-26,  5.2579e-08,  ...,  1.2914e-32,
          1.3958e-04,  4.3785e-05],
        [-6.9562e-04,  1.2511e-25,  2.0401e-13,  ...,  7.6237e-19,
          3.1589e-12,  1.4696e-14],
        [-6.9039e-04,  1.7033e-35,  4.7859e-20,  ...,  2.5736e-31,
          1.9125e-16,  5.3670e-09],
        ...,
        [ 6.6639e-26,  1.6557e-21, -1.0157e-16,  ...,  6.7143e-29,
          5.6643e-31,  1.0157e-16],
        [ 3.9409e-17,  2.4297e-28,  1.0603e-23,  ..., -3.9409e-17,
          5.4400e-35,  1.1821e-38],
        [ 5.3087e-15, -2.7769e-08,  4.8879e-09,  ...,  8.0249e-18,
          1.8919e-21,  2.2881e-08]])

In [52]:
# スクラッチのdL/dZ2の結果
Z2.grad_

tensor([[ 7.5433e-08,  1.5461e-26,  5.2579e-08,  ...,  1.2914e-32,
          1.3958e-04,  4.3785e-05],
        [-6.9562e-04,  1.2511e-25,  2.0401e-13,  ...,  7.6237e-19,
          3.1589e-12,  1.4696e-14],
        [-6.9589e-04,  1.7169e-35,  4.8241e-20,  ...,  2.5941e-31,
          1.9277e-16,  5.4098e-09],
        ...,
        [ 4.5659e-13,  1.1344e-08, -6.9589e-04,  ...,  4.6004e-16,
          3.8809e-18,  6.9588e-04],
        [ 6.9589e-04,  4.2904e-15,  1.8723e-10,  ..., -6.9589e-04,
          9.6061e-22,  2.0874e-25],
        [ 5.3087e-15, -2.7791e-08,  4.8879e-09,  ...,  8.0249e-18,
          1.8919e-21,  2.2881e-08]], grad_fn=<DivBackward0>)

In [54]:
# torch.allcloseで大体同じか否かを判定できる。
torch.allclose(Z2.grad, Z2.grad_)

False

In [65]:
Z2

tensor([[ 10.5779, -32.4535,  10.2170,  ..., -46.4491,  18.1011,  16.9417],
        [ 24.0194, -18.1953,   9.9247,  ...,  -2.5726,  12.6645,   7.2941],
        [ 22.3373, -32.2485,   3.3233,  ..., -22.6255,  11.6164,  28.7664],
        ...,
        [  5.0758,  15.1963, -26.3609,  ...,  -1.8244,  -6.5997,  26.2205],
        [ 38.9531,  13.1410,  23.8248,  ..., -14.5750,  -2.1711, -10.6053],
        [ -0.3404,  25.2586,  13.3925,  ...,  -6.8350, -15.1877,  14.9360]],
       grad_fn=<AddBackward0>)

Z2の勾配を計算すると、  
スクラッチとautogradでデータによって同じだったり、異なったりしていることが分かる。  

深層学習の問題点の一つとして、Xに始まる入力に対して各層で線形結合を繰り返すと    
出力結果が元のXの分布からどんどん大きくなっていき、 大きくなり過ぎた出力を最終的にsoftmax関数へ入れることで、  
eの累乗の計算が大きくなり過ぎて勾配計算に誤差が出るという点がある。  
上記におけるsoftmax関数の入力はZ2であり、Z2を見ると指数としては大きい値になっていることが分かる。

上記ではsoftmax関数への入力が非常に大きい場合の取り扱い方が、  
スクラッチ実装とpytorchの.backward()の実装で恐らく違いがあるため、上記のようにところどころ計算が合わなくなったのだろう。  

## ■ 一致しないことに対する対処
出力が大きくなり過ぎないための工夫として、  
重みの初期化の際にrandn（標準正規分布からのサンプリング）ではなく、  
平均0, 分散$\frac{2}{n}$(nは特徴量数) の正規分布から取り出すというものが知られている。（下記）  
なぜこれでうまくいくのかは後述。  

In [68]:
# データ数、特徴量数、隠れ層のノード数、最終的な出力数（ここではクラス数）を定義
m, nf = X_train.shape
nh = 30
n_out = 10

# 入力層→隠れ層の重みW、バイアス項bの初期化
# W_1 = torch.randn(size=(nh, nf) ,requires_grad=True) #出力×入力

# randnは分散が1なので、2/nのルートをかければよい。
W_1 = torch.randn((nh, nf)) * torch.sqrt(torch.tensor(2./nf))
W_1.requires_grad = True # 一つ上のrandnでTrueにしても計算後のtensorに引き継がれないので別途設定する必要あり。
B_1 = torch.zeros(size=(1, nh), requires_grad=True) # 1 x 出力

# 隠れ層→出力層の重みW、バイアス項bの初期化
#W_2 = torch.randn(size=(n_out, nh) ,requires_grad=True) #出力×入力
W_2 = torch.randn((n_out, nh)) * torch.sqrt(torch.tensor(2./nf))
W_2.requires_grad = True 

B_2 = torch.zeros(size=(1, n_out), requires_grad=True) # 1 x 出力
W_list = [W_1, W_2]
B_list = [B_1, B_2]


In [69]:
# 上記の工夫を踏まえて再計算
loss, Z1, A1, Z2, A2 = forward_and_backward(X_train, W_list=W_list, B_list=B_list, y=y_train)
loss.backward()

In [75]:
# autogradの結果
W_1.grad

tensor([[-0.0161, -0.0156, -0.0057,  ...,  0.0128, -0.0099, -0.0163],
        [ 0.0014,  0.0016,  0.0030,  ..., -0.0009,  0.0004,  0.0004],
        [-0.0028, -0.0033, -0.0036,  ...,  0.0017, -0.0010, -0.0014],
        ...,
        [-0.0020, -0.0006,  0.0047,  ...,  0.0022, -0.0012, -0.0021],
        [-0.0010, -0.0025,  0.0062,  ...,  0.0099, -0.0066, -0.0058],
        [-0.0045, -0.0044, -0.0037,  ...,  0.0075,  0.0002, -0.0043]])

In [76]:
# スクラッチ実装の結果
W_1.grad_

tensor([[-0.0161, -0.0156, -0.0057,  ...,  0.0128, -0.0099, -0.0163],
        [ 0.0014,  0.0016,  0.0030,  ..., -0.0009,  0.0004,  0.0004],
        [-0.0028, -0.0033, -0.0036,  ...,  0.0017, -0.0010, -0.0014],
        ...,
        [-0.0020, -0.0006,  0.0047,  ...,  0.0022, -0.0012, -0.0021],
        [-0.0010, -0.0025,  0.0062,  ...,  0.0099, -0.0066, -0.0058],
        [-0.0045, -0.0044, -0.0037,  ...,  0.0075,  0.0002, -0.0043]],
       grad_fn=<MmBackward0>)

In [79]:
# autogradと大体等しいことを確認
print(torch.allclose(W_1.grad_, W_1.grad))
print(torch.allclose(B_1.grad_, B_1.grad))
print(torch.allclose(W_2.grad_, W_2.grad))
print(torch.allclose(B_2.grad_, B_2.grad))
print(torch.allclose(Z1.grad_, Z1.grad))
print(torch.allclose(Z1.grad_, Z1.grad))


True
True
True
True
True
True


工夫前と異なり、autogradと結果が等しくなった。

In [70]:
Z2

tensor([[ 0.6026, -2.2660,  0.7763,  ..., -0.6327, -1.6516, -0.1160],
        [ 0.4614, -1.5669,  0.4437,  ..., -0.1522, -1.4070, -0.6645],
        [-0.3714, -1.2673,  0.8341,  ..., -0.4784, -0.6906, -1.3658],
        ...,
        [-0.2987, -2.2417,  1.4166,  ..., -1.0039, -1.0017, -1.0237],
        [-0.9527, -0.4589,  0.8461,  ..., -0.3010,  0.7702, -1.7446],
        [ 0.5703, -1.2861,  2.1643,  ..., -0.8342,  0.0096, -1.9183]],
       grad_fn=<AddBackward0>)

工夫の結果、softmaxへの入力Z2がそれほど大きくならなくなったことが分かる。